### Parâmetros para modificação:

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from random import randint
from sklearn.manifold import TSNE
from scipy.stats import randint
# Evitar warnings:
import warnings
warnings.filterwarnings("ignore")

In [2]:
CUT_OFF_DATE = pd.to_datetime('2022-12-31')
DATE_OF_INFERENCE = '2023-11-17'
# Definindo dt como 90 dias
dt = pd.Timedelta('90 days')

### Variáveis atemporais:

In [3]:
# Substitua o caminho das bases aqui:
subscriptions_features_dataset_filepath = '../Tabelas/subscriptions_features_dataset.csv'
tendencia_de_utilizacao_filepath = '../Tabelas/tendência_de_utilização.csv'
features_filepath = '../Tabelas/features.csv'

x1 = pd.read_csv(subscriptions_features_dataset_filepath)
x5 = pd.read_csv(features_filepath)

In [5]:
df_final = pd.merge(x1, x5, on='employee_id', how='left')
print(x1.shape)
print(x5.shape)
print(df_final.shape)
display(df_final.head())
# Selecionar colunas para clusterização
col_cluster = ['employee_id','type_mapped', 'gender_maped', 
               'Binario_fee', 'Fee', 'NumGymsWithinRadius', 
               'NumGymsNearCompany', 'distancia_cliente_empresa',
               'payment_source_y']
df_atemp = df_final[col_cluster]
df_atemp['employee_id'] = df_atemp['employee_id'].astype(int) 
display(df_atemp)

'''
Atemporais: type mapped, gender maped, binario fee, fee, num gyns within radius,
num gyms near company, distancia cliente empresa, payment source

Temporais: upgrade, age, count subscriptions, numero de ids distintos, normalized stats, 
gym status, utilizacoes, pref_modality, num_distinct_gyms, uses per week, most frequent hour
'''

(1605146, 24)
(1013944, 10)
(1764656, 33)


,subscription_id,employee_id,company_id,company_plan_id,country_id,created_at,started_at,suspended_at,canceled_at,subscription_status,...,distancia_cliente_empresa,gym_id,gym_status,utilizacoes,pref_modality,num_main_modality,num_distinct_gyms,payment_source_y,most_frequent_hour,uses_per_week
0,110,130,45,76,2,2017-06-23 21:04:17.923,2017-06-23 00:00:00.000,NaN,2017-09-23 23:59:59,cancelled,...,12.876877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,142,162,45,76,2,2017-06-23 21:04:24.686,2017-06-23 00:00:00.000,NaN,2017-09-23 23:59:59,cancelled,...,5.561663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,261,28163,57,116,2,2017-08-18 14:28:51.907,2017-08-18 00:00:00.000,NaN,2018-03-18 23:59:59,cancelled,...,2.580917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,333,21739,57,116,2,2017-08-21 13:57:45.665,2017-08-21 00:00:00.000,NaN,2018-11-21 23:59:59,cancelled,...,874.545486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,398,93992,57,117,2,2017-08-22 11:45:32.009,2017-08-22 00:00:00.000,NaN,2018-07-22 23:59:59,cancelled,...,8.854492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,employee_id,type_mapped,gender_maped,Binario_fee,Fee,NumGymsWithinRadius,NumGymsNearCompany,distancia_cliente_empresa,payment_source_y
0,130,0.0,NaN,0.0,0.0,211.0,505,12.876877,NaN
1,162,0.0,NaN,1.0,420.0,309.0,505,5.561663,NaN
2,28163,0.0,NaN,0.0,0.0,245.0,217,2.580917,NaN
3,21739,0.0,NaN,0.0,0.0,14.0,217,874.545486,NaN
4,93992,0.0,NaN,1.0,300.0,537.0,217,8.854492,NaN
...,...,...,...,...,...,...,...,...,...
1764651,935208,0.0,1.0,NaN,NaN,45.0,56,3.391813,prepaid
1764652,2006364,1.0,0.0,NaN,NaN,177.0,500,9.818316,NaN
1764653,2087520,1.0,1.0,NaN,NaN,1.0,271,16.939099,prepaid
1764654,2004863,1.0,1.0,NaN,NaN,185.0,500,9.887746,NaN


'\nAtemporais: type mapped, gender maped, binario fee, fee, num gyns within radius,\nnum gyms near company, distancia cliente empresa, payment source\n\nTemporais: upgrade, age, count subscriptions, numero de ids distintos, normalized stats, \ngym status, utilizacoes, pref_modality, num_distinct_gyms, uses per week, most frequent hour\n'

### Variáveis Temporais

In [6]:
# Penamá
def calcular_gym_status(CUT_OFF_DATE,df):
    today = CUT_OFF_DATE
    df['validated_at'] = pd.to_datetime(df['validated_at'])
    df['created_at'] = pd.to_datetime(df['created_at'])
    df = df.drop('Unnamed: 0', axis=1)
    df1 = df[df['validated_at'] <= today]
    df2 = df1[['validated_at','employee_id','token','gym_id']]
    academias_mais_utilizadas = df2.groupby(['employee_id', 'gym_id']).size().reset_index(name='utilizacoes')
    idx = academias_mais_utilizadas.groupby(['employee_id'])['utilizacoes'].transform(max) == academias_mais_utilizadas['utilizacoes']
    academias_mais_utilizadas = academias_mais_utilizadas[idx]
    gyms = pd.read_csv('C:\\Users\\Pedro Nakandakari\\Downloads\\Total Pass\\gyms.csv')
    gyms = gyms.drop('Unnamed: 0', axis=1)
    gyms['registered_at'] = pd.to_datetime(gyms['registered_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
    gyms['activated_at'] = pd.to_datetime(gyms['activated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
    gyms['last_inactivated_at'] = pd.to_datetime(gyms['last_inactivated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
    gyms1 = gyms[gyms['registered_at'] <= today]
    gyms2 = gyms1[['gym_id','gym_status']]
    dfprinc = pd.merge(gyms2, academias_mais_utilizadas, on='gym_id', how='right')
    return dfprinc
    

In [7]:
from datetime import datetime, date
def calcular_age(CUT_OFF_DATE):
    df2 = pd.read_csv('../Tabelas/employees.csv')

    df2['birthdate'] = pd.to_datetime(df2['birthdate'], errors='coerce')
    df2 = df2.dropna(subset=['birthdate'])

    today = CUT_OFF_DATE
    df2 = df2[['employee_id', 'birthdate']]
    # iterando o df2 calculando a coluna 'age'
    df2['age'] = df2.apply(lambda row: today.year - row['birthdate'].year - ((today.month < row['birthdate'].month) or ((today.month == row['birthdate'].month) and (today.day < row['birthdate'].day))), axis=1)
    df2 = df2[['employee_id','age']]
    return df2

In [8]:
# Gurgel
def calcular_freezes(CUT_OFF_DATE):
    df1 = pd.read_csv('../Tabelas/freezes.csv')
    df2 = pd.read_csv('../Tabelas/subscriptions.csv')


    distinct_ids_counts = df1.groupby('employee_id')['id'].nunique().reset_index()
    distinct_ids_counts.columns = ['employee_id', 'freezes']

    df2 = pd.merge(df2, distinct_ids_counts, on='employee_id', how='left')
    df2['freezes'].fillna(0, inplace=True)

    df2 = df2[['employee_id', 'freezes']]
    
    return df2

In [9]:
# Gurgel
def calcular_upgrade(CUT_OFF_DATE):
    df2 = pd.read_csv('../Tabelas/subscriptions.csv')

    df2['upgrade'] = np.where(df2['migrated_from_subscription_id'].notna(), 1, 0)

    filtered_subscriptions = df2[df2['upgrade'] == 0]

    count_subscriptions_by_employee = filtered_subscriptions.groupby('employee_id')['subscription_id'].count().reset_index()

    count_subscriptions_by_employee = count_subscriptions_by_employee.rename(columns={'subscription_id': 'N° Subscriptions'})

    df2 = pd.merge(df2, count_subscriptions_by_employee, on='employee_id', how='left')

    df2 = df2[['employee_id', 'upgrade','N° Subscriptions']]

    return df2

In [10]:
# Pra depois
def calcular_normalized_stats(CUT_OFF_DATE):

SyntaxError: incomplete input (417630146.py, line 2)

In [10]:
# Penamá
def calcular_pref_modality_utilizaçoes(CUT_OFF_DATE,df):
    today = CUT_OFF_DATE
    df['validated_at'] = pd.to_datetime(df['validated_at'])
    df['created_at'] = pd.to_datetime(df['created_at'])
    df = df.drop('Unnamed: 0', axis=1)
    df1 = df[df['validated_at'] <= today]
    gyms = pd.read_csv('C:\\Users\\Pedro Nakandakari\\Downloads\\Total Pass\\gyms.csv')
    gyms = gyms.drop('Unnamed: 0', axis=1)
    gyms['registered_at'] = pd.to_datetime(gyms['registered_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
    gyms['activated_at'] = pd.to_datetime(gyms['activated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
    gyms['last_inactivated_at'] = pd.to_datetime(gyms['last_inactivated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
    gyms1 = gyms[gyms['registered_at'] <= today]
    df2 = df1[['employee_id','gym_id']]
    gyms2 = gyms1[['gym_id','main_modality']]
    gymsdf = pd.merge(gyms2, df2, how= 'right', on= ['gym_id'])
    modalidades_preferidas = gymsdf.groupby(['employee_id', 'main_modality']).size().reset_index(name='num_main_modality')
    idx1 = modalidades_preferidas.groupby(['employee_id'])['num_main_modality'].transform(max) == modalidades_preferidas['num_main_modality']
    modalidades_preferidas = modalidades_preferidas[idx1]
    modalidades_preferidas = modalidades_preferidas.rename(columns={'main_modality': 'pref_modality'})
    gymsprinc = pd.merge(gymsdf, modalidades_preferidas[['employee_id', 'pref_modality', 'num_main_modality']],
              on=['employee_id'], how='left')
    return gymsprinc

In [11]:
# Penamá
def calcular_num_distinct_gyms(CUT_OFF_DATE,df):
    today = CUT_OFF_DATE
    df1 = df[df['validated_at'] <= today]
    df1['validated_at'] = pd.to_datetime(df1['validated_at'])
    df1['created_at'] = pd.to_datetime(df1['created_at'])
    df1 = df1.drop('Unnamed: 0', axis=1)
    df2 = df1[['employee_id','gym_id']]
    academias_distintas_por_usuario = df2.groupby('employee_id')['gym_id'].nunique().reset_index(name='num_distinct_gyms')
    distpref = pd.merge(df2, academias_distintas_por_usuario, on='employee_id', how='left')
    return distpref

In [12]:
# Penamá
def calcular_uses_per_week(CUT_OFF_DATE,df):
    df['validated_at_minute'] = pd.to_datetime(df['validated_at_minute'])
    df1 = df[['employee_id','validated_at_minute']]    
    df1['Ano'] = df1['validated_at_minute'].dt.year
    df1['Semana'] = df1['validated_at_minute'].dt.isocalendar().week
    today = CUT_OFF_DATE
    df2 = df1[df1['validated_at_minute'] <= today]
    num_utilizacoes_por_semana = df1.groupby(['employee_id', 'Ano', 'Semana']).size().reset_index(name='Num_Utilizacoes')
    media_utilizacoes_por_semana = num_utilizacoes_por_semana.groupby('employee_id')['Num_Utilizacoes'].mean().reset_index(name='uses_per_week')
    df3 = pd.merge(df2, media_utilizacoes_por_semana, how= 'left', on= ['employee_id'])
    return df3

In [13]:
# Penamá
def calcular_most_frequent_hour(CUT_OFF_DATE,df):
    df['validated_at_minute'] = pd.to_datetime(df['validated_at_minute'])
    df1 = df[['employee_id','validated_at_minute']]
    today = CUT_OFF_DATE
    df2 = df1[df1['validated_at_minute'] <= today]
    hora_mais_frequente_por_usuario = df2.groupby('employee_id')['validated_at_minute'].agg(lambda x: x.dt.hour.mode().iloc[0])
    df3 = pd.merge(df2, hora_mais_frequente_por_usuario, on='employee_id', how='left')
    return df3

In [14]:
# Bernardo
def active_at_date(CUT_OFF_DATE):
    df2 = pd.read_csv('../Tabelas/subscriptions.csv')

    df2['created_at'] = pd.to_datetime(df2['created_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
    df2['canceled_at'] = pd.to_datetime(df2['canceled_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
    
    # Selecionando 1 apenas as assinaturas ativas na data de corte
    df2['active_at_date'] = ((df2['created_at'] <= CUT_OFF_DATE) & ((df2['canceled_at'] >= CUT_OFF_DATE) | (df2['canceled_at'].isnull()))).astype(int)
    # Para ser ativo,
    # 1. A assinatura deve ter sido criada antes da data de corte
    # 2. A assinatura deve ter sido cancelada depois da data de corte ou não ter sido cancelada

    df2 = df2[['employee_id','active_at_date']]
    return df2

In [15]:
# Bernardo - cancelou no gap
def churn_at_dt(CUT_OFF_DATE,dt):
    df2 = pd.read_csv('../Tabelas/subscriptions.csv')

    df2['created_at'] = pd.to_datetime(df2['created_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
    df2['canceled_at'] = pd.to_datetime(df2['canceled_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
    
    # Selecionando 1 apenas as assinaturas ativas na data de corte
    df2['churn_at_dt'] = ((df2['created_at'] <= CUT_OFF_DATE) & ((df2['canceled_at'] >= CUT_OFF_DATE) & (df2['canceled_at'] <= CUT_OFF_DATE + dt))).astype(int)
    # Para dar churn no gap,
    # 1. A assinatura deve ter sido criada antes da data de corte
    # 2. A assinatura deve ter sido cancelada depois da data de corte e antes da data de corte + dt

    df2 = df2[['employee_id','churn_at_dt']]
    return df2

In [16]:
# Carregando todas as funções com CUT_OFF_DATE
df_age = calcular_age(CUT_OFF_DATE)
df_freezes = calcular_freezes(CUT_OFF_DATE)
df_upgrade = calcular_upgrade(CUT_OFF_DATE)

In [32]:
df_token = pd.read_csv('../Tabelas/daily_token_validation-001.csv')

: 

In [20]:
df_gym_status = calcular_gym_status(CUT_OFF_DATE,df_token)
df_pref_modality = calcular_pref_modality_utilizaçoes(CUT_OFF_DATE,df_token)
df_num_distinct_gyms = calcular_num_distinct_gyms(CUT_OFF_DATE,df_token)
df_uses_per_week = calcular_uses_per_week(CUT_OFF_DATE,df_token)
df_most_frequent_hour = calcular_most_frequent_hour(CUT_OFF_DATE,df_token)

: 

In [17]:
df_active_at_date = active_at_date(CUT_OFF_DATE)
df_churn_at_dt = churn_at_dt(CUT_OFF_DATE, dt)

In [18]:
# Merge nos 3 DataFrames com chave 'employee_id'
df_temp = pd.merge(df_age, df_upgrade, on='employee_id', how='inner')
print("Após o primeiro merge:", df_temp.shape)

# Remove duplicatas:
df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
print("Após remover duplicatas:", df_temp.shape)

df_temp = pd.merge(df_temp, df_freezes, on='employee_id', how='inner')
print("Após o segundo merge:", df_temp.shape)

# Remove duplicatas:
df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
print("Após remover duplicatas:", df_temp.shape)

df_temp = pd.merge(df_temp, df_active_at_date, on='employee_id', how='inner')
print("Após o terceiro merge:", df_temp.shape)

# Remove duplicatas:
df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
print("Após remover duplicatas:", df_temp.shape)

df_temp = pd.merge(df_temp, df_churn_at_dt, on='employee_id', how='right')
print("Após o quarto merge:", df_temp.shape)

# Remove duplicatas:
df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
print("Após remover duplicatas:", df_temp.shape)

display(df_temp)

Após o primeiro merge: (1558762, 4)
Após remover duplicatas: (1099154, 4)
Após o segundo merge: (1558762, 5)
Após remover duplicatas: (1099154, 5)
Após o terceiro merge: (1558762, 6)
Após remover duplicatas: (1099154, 6)
Após o quarto merge: (1605146, 7)
Após remover duplicatas: (1136775, 7)


,employee_id,age,upgrade,N° Subscriptions,freezes,active_at_date,churn_at_dt
0,130.0,37.0,0.0,2.0,0.0,0.0,0
1,162.0,32.0,0.0,1.0,0.0,0.0,0
2,28163.0,47.0,0.0,1.0,0.0,0.0,0
3,21739.0,35.0,0.0,1.0,0.0,0.0,0
4,93992.0,37.0,0.0,2.0,0.0,0.0,0
...,...,...,...,...,...,...,...
1605139,2091820.0,36.0,0.0,1.0,0.0,0.0,0
1605140,2091851.0,28.0,0.0,1.0,0.0,0.0,0
1605142,2006364.0,37.0,0.0,1.0,0.0,0.0,0
1605144,2004863.0,37.0,0.0,1.0,0.0,0.0,0


### Juntando variáveis temporais com atemporais:

In [19]:
# merging df com df_atemp
print("Antes do merge com df_atemp:", df_temp.shape)
df = pd.merge(df_temp, df_atemp, on='employee_id', how='left')
print("Após o merge com df_atemp:", df.shape)
# remove duplicatas
df = df.drop_duplicates(subset='employee_id',keep='first')
print("Após remover duplicatas:", df.shape)
df 

Antes do merge com df_atemp: (1136775, 7)
Após o merge com df_atemp: (1764656, 15)
Após remover duplicatas: (1136775, 15)


,employee_id,age,upgrade,N° Subscriptions,freezes,active_at_date,churn_at_dt,type_mapped,gender_maped,Binario_fee,Fee,NumGymsWithinRadius,NumGymsNearCompany,distancia_cliente_empresa,payment_source_y
0,130.0,37.0,0.0,2.0,0.0,0.0,0,0.0,NaN,0.0,0.0,211.0,505,12.876877,NaN
3,162.0,32.0,0.0,1.0,0.0,0.0,0,0.0,NaN,1.0,420.0,309.0,505,5.561663,NaN
5,28163.0,47.0,0.0,1.0,0.0,0.0,0,0.0,NaN,0.0,0.0,245.0,217,2.580917,NaN
6,21739.0,35.0,0.0,1.0,0.0,0.0,0,0.0,NaN,0.0,0.0,14.0,217,874.545486,NaN
7,93992.0,37.0,0.0,2.0,0.0,0.0,0,0.0,NaN,1.0,300.0,537.0,217,8.854492,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764651,2091820.0,36.0,0.0,1.0,0.0,0.0,0,1.0,1.0,NaN,NaN,224.0,0,NaN,NaN
1764652,2091851.0,28.0,0.0,1.0,0.0,0.0,0,1.0,1.0,NaN,NaN,131.0,353,18.229972,NaN
1764653,2006364.0,37.0,0.0,1.0,0.0,0.0,0,1.0,0.0,NaN,NaN,177.0,500,9.818316,NaN
1764654,2004863.0,37.0,0.0,1.0,0.0,0.0,0,1.0,1.0,NaN,NaN,185.0,500,9.887746,NaN


### Pré processamento dos dados

In [20]:
df.isnull().sum()

employee_id                        0
age                            37621
upgrade                        37621
N° Subscriptions               37621
freezes                        37621
active_at_date                 37621
churn_at_dt                        0
type_mapped                        1
gender_maped                   55594
Binario_fee                  1127071
Fee                          1127071
NumGymsWithinRadius                1
NumGymsNearCompany                 0
distancia_cliente_empresa      94195
payment_source_y              202276
dtype: int64

In [21]:
df

,employee_id,age,upgrade,N° Subscriptions,freezes,active_at_date,churn_at_dt,type_mapped,gender_maped,Binario_fee,Fee,NumGymsWithinRadius,NumGymsNearCompany,distancia_cliente_empresa,payment_source_y
0,130.0,37.0,0.0,2.0,0.0,0.0,0,0.0,NaN,0.0,0.0,211.0,505,12.876877,NaN
3,162.0,32.0,0.0,1.0,0.0,0.0,0,0.0,NaN,1.0,420.0,309.0,505,5.561663,NaN
5,28163.0,47.0,0.0,1.0,0.0,0.0,0,0.0,NaN,0.0,0.0,245.0,217,2.580917,NaN
6,21739.0,35.0,0.0,1.0,0.0,0.0,0,0.0,NaN,0.0,0.0,14.0,217,874.545486,NaN
7,93992.0,37.0,0.0,2.0,0.0,0.0,0,0.0,NaN,1.0,300.0,537.0,217,8.854492,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764651,2091820.0,36.0,0.0,1.0,0.0,0.0,0,1.0,1.0,NaN,NaN,224.0,0,NaN,NaN
1764652,2091851.0,28.0,0.0,1.0,0.0,0.0,0,1.0,1.0,NaN,NaN,131.0,353,18.229972,NaN
1764653,2006364.0,37.0,0.0,1.0,0.0,0.0,0,1.0,0.0,NaN,NaN,177.0,500,9.818316,NaN
1764654,2004863.0,37.0,0.0,1.0,0.0,0.0,0,1.0,1.0,NaN,NaN,185.0,500,9.887746,NaN


In [22]:
# dropar nulos com base na coluna 'active_at_date'
df = df.dropna(subset=['active_at_date'])

# Substitua active por 1 e inactive por 0 em gym status:
#df['gym_status'] = df['gym_status'].map({'active': 1, 'inactive': 0})
# Substitua musculação por 1 e qualquer outra categoria por 0 em pref modality:
#df['pref_modality'] = df['pref_modality'].map(lambda x: 1 if x == 'MUSCULAÇÃO' else 0)
# Substitua payroll por 0, prepaid por 1 e own_wallet por 2 em payment source:
df['payment_source_y'] = df['payment_source_y'].map({'payroll_deduction': 0, 'prepaid': 1, 'own_wallet': 2,})

# Preenchendo valores nulos com 0
df['Binario_fee'].fillna(0, inplace=True)
df['Fee'].fillna(0, inplace=True)
df['freezes'].fillna(0, inplace=True)
df['upgrade'].fillna(0, inplace=True)
df['N° Subscriptions'].fillna(0, inplace=True)
df['payment_source_y'].fillna(0, inplace=True)
#df['pref_modality'].fillna(1, inplace=True) # Preenche com musculação

# Preenchendo valores nulos com a média
media_distancia = df['distancia_cliente_empresa'].mean()
media_idade = df['age'].mean()
media_genero = df['gender_maped'].mean()

# Preenche os valores NaN com as médias calculadas
df['distancia_cliente_empresa'].fillna(media_distancia, inplace=True)
df['age'].fillna(media_idade, inplace=True)
df['gender_maped'].fillna(media_genero, inplace=True)
df.dropna(subset=['type_mapped', 'NumGymsWithinRadius'], inplace=True)

df.isnull().sum()

employee_id                  0
age                          0
upgrade                      0
N° Subscriptions             0
freezes                      0
active_at_date               0
churn_at_dt                  0
type_mapped                  0
gender_maped                 0
Binario_fee                  0
Fee                          0
NumGymsWithinRadius          0
NumGymsNearCompany           0
distancia_cliente_empresa    0
payment_source_y             0
dtype: int64

In [23]:
# Filtre os dados para que apenas os usuários ativos sejam considerados
df = df[df['active_at_date'] == 1]
df

,employee_id,age,upgrade,N° Subscriptions,freezes,active_at_date,churn_at_dt,type_mapped,gender_maped,Binario_fee,Fee,NumGymsWithinRadius,NumGymsNearCompany,distancia_cliente_empresa,payment_source_y
43,9834.0,50.0,0.0,1.0,2.0,1.0,0,0.0,0.477098,0.0,0.0,0.0,547,322.891290,0.0
165,61586.0,42.0,0.0,1.0,0.0,1.0,0,0.0,0.000000,0.0,0.0,1.0,217,1965.192265,0.0
204,26531.0,36.0,0.0,1.0,32.0,1.0,0,0.0,0.477098,0.0,0.0,0.0,547,322.891290,0.0
240,237043.0,28.0,0.0,1.0,0.0,1.0,1,0.0,0.000000,0.0,0.0,20.0,87,853.320704,0.0
267,229834.0,40.0,0.0,1.0,0.0,1.0,0,0.0,0.000000,1.0,250.0,16.0,87,396.939346,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757781,1292983.0,38.0,0.0,1.0,0.0,1.0,0,0.0,0.000000,0.0,0.0,37.0,59,10.670243,0.0
1757782,1131868.0,34.0,0.0,1.0,0.0,1.0,1,0.0,1.000000,0.0,0.0,0.0,47,322.891290,0.0
1757783,1293468.0,32.0,0.0,1.0,0.0,1.0,0,0.0,0.000000,0.0,0.0,125.0,472,290.004726,1.0
1757784,1293375.0,24.0,0.0,1.0,0.0,1.0,0,0.0,1.000000,0.0,0.0,0.0,169,29.315979,0.0


### Previsão

In [24]:
# Faça um modelo de regressão logística que dê a probabilidade do churn_at_dt ser 1
# A variável resposta é churn_at_dt
# As variáveis preditoras são todas as outras
# A variável resposta é binária, então é um problema de classificação binária
# Vamos usar a função train_test_split para dividir os dados em treino e teste
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Definindo as variáveis preditoras e a variável resposta
X = df.drop(['employee_id', 'churn_at_dt','active_at_date'], axis=1)
# Normalizando os dados
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
y = df['churn_at_dt']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Instanciando o modelo
logreg = LogisticRegression()

# Treinando o modelo
logreg.fit(X_train, y_train)

# Fazendo previsões
y_pred = logreg.predict(X_test)

# Avaliando o modelo
print('Acurácia:', accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Acurácia: 0.7046898742920293
[[40503   256]
 [16846   307]]
              precision    recall  f1-score   support

           0       0.71      0.99      0.83     40759
           1       0.55      0.02      0.03     17153

    accuracy                           0.70     57912
   macro avg       0.63      0.51      0.43     57912
weighted avg       0.66      0.70      0.59     57912



In [25]:
# Use undersampling para balancear as classes presentes na variável resposta
# A variável resposta é binária, então é um problema de classificação binária
# Vamos usar a função train_test_split para dividir os dados em treino e teste
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Definindo as variáveis preditoras e a variável resposta
X = df.drop(['employee_id', 'churn_at_dt','active_at_date'], axis=1)
# Normalizando os dados
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
y = df['churn_at_dt']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Instanciando o modelo
logreg = LogisticRegression()

# Instanciando o objeto RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

# Aplicando o undersampling
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

# Treinando o modelo
logreg.fit(X_resampled, y_resampled)

# Fazendo previsões
y_pred = logreg.predict(X_test)

# Avaliando o modelo
print('Acurácia:', accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Acurácia: 0.5779803840309435
[[24892 15867]
 [ 8573  8580]]
              precision    recall  f1-score   support

           0       0.74      0.61      0.67     40759
           1       0.35      0.50      0.41     17153

    accuracy                           0.58     57912
   macro avg       0.55      0.56      0.54     57912
weighted avg       0.63      0.58      0.59     57912



In [26]:
# Use oversampling para balancear as classes presentes na variável resposta
# A variável resposta é binária, então é um problema de classificação binária
# Vamos usar a função train_test_split para dividir os dados em treino e teste
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Definindo as variáveis preditoras e a variável resposta
X = df.drop(['employee_id', 'churn_at_dt','active_at_date'], axis=1)
# Normalizando os dados
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
y = df['churn_at_dt']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Instanciando o modelo
logreg = LogisticRegression()

# Instanciando o objeto RandomUnderSampler

ros = RandomOverSampler(random_state=42)

# Aplicando o undersampling
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

# Treinando o modelo
logreg.fit(X_resampled, y_resampled)

# Fazendo previsões
y_pred = logreg.predict(X_test)

# Avaliando o modelo
print('Acurácia:', accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Acurácia: 0.5779803840309435
[[24892 15867]
 [ 8573  8580]]
              precision    recall  f1-score   support

           0       0.74      0.61      0.67     40759
           1       0.35      0.50      0.41     17153

    accuracy                           0.58     57912
   macro avg       0.55      0.56      0.54     57912
weighted avg       0.63      0.58      0.59     57912



### Exemplo de Inferência

In [47]:
# exemplo de uma inferência no logreg
AGE = 40
UPGRADE = 0
FREEZES = 0
N_SUBSCRIPTIONS = 1
TYPE_MAPPED = 0
GENDER_MAPED = 1
BINARIO_FEE = 0
FEE = 1
NUM_GYM_WITHIN_RADIUS = 1
NUM_GYM_NEAR_COMPANY = 1
DISTANCIA_CLIENTE_EMPRESA = 100
PAYMENT_SOURCE_Y = 2

# Agrupe todas as variáveis
X_inferencia = [[AGE, UPGRADE, FREEZES, N_SUBSCRIPTIONS, TYPE_MAPPED, GENDER_MAPED, BINARIO_FEE, FEE, NUM_GYM_WITHIN_RADIUS, NUM_GYM_NEAR_COMPANY, DISTANCIA_CLIENTE_EMPRESA, PAYMENT_SOURCE_Y]]

# Normalizando os dados
X_inferencia = scaler.transform(X_inferencia)

# Faça a previsão com a probabilidade do churn, indicando qual é qual
print('Chance do cliente dar Churn: ', round(logreg.predict_proba(X_inferencia)[0][1],4)* 100,'%')

Chance do cliente dar Churn:  44.93 %
